In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.master("local[4]").appName("df -giris").config("spark.executor.memory","4g").config("spark.driver.memory","2g").getOrCreate()

In [3]:
sc=spark.sparkContext

In [4]:
##listeler df

from pyspark.sql import Row
list_rdd=sc.parallelize([1,2,3,4,5,6,5,4]).map(lambda x: Row(x))

In [5]:
dff_from_list=list_rdd.toDF(["rakamlar"])

In [6]:
dff_from_list.show()

+--------+
|rakamlar|
+--------+
|       1|
|       2|
|       3|
|       4|
|       5|
|       6|
|       5|
|       4|
+--------+



In [7]:
#range ile df

df_from_range=sc.parallelize(range(10,100,5)).map(lambda x:(x,)).toDF(["range"])
df_from_range.show(3)

+-----+
|range|
+-----+
|   10|
|   15|
|   20|
+-----+
only showing top 3 rows



In [8]:
#yöntem2

from pyspark.sql.types import IntegerType
dff_from_range2=spark.createDataFrame(range(10,100,5),IntegerType())
dff_from_range2

DataFrame[value: int]

In [9]:
#dosyadan df
df_from_file=spark.read.csv("C:\\Users\ege_s\\Desktop\\pyspark\\OnlineRetail.csv")
df_from_file.show()

+--------------------+
|                 _c0|
+--------------------+
|InvoiceNo;StockCo...|
|536365;85123A;WHI...|
|536365;71053;WHIT...|
|536365;84406B;CRE...|
|536365;84029G;KNI...|
|536365;84029E;RED...|
|536365;22752;SET ...|
|536365;21730;GLAS...|
|536366;22633;HAND...|
|536366;22632;HAND...|
|536367;84879;ASSO...|
|536367;22745;POPP...|
|536367;22748;POPP...|
|536367;22749;FELT...|
|536367;22310;IVOR...|
|536367;84969;BOX ...|
|536367;22623;BOX ...|
|536367;22622;BOX ...|
|536367;21754;HOME...|
|536367;21755;LOVE...|
+--------------------+
only showing top 20 rows



In [10]:
df_from_file=spark.read.option("sep",";").option("header","True").option("inferSchema","True").csv("C:\\Users\ege_s\\Desktop\\pyspark\\OnlineRetail.csv")

In [11]:
df_from_file.show(3)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 3 rows



In [12]:
df_from_file.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [13]:
df_from_file.count()

541909

In [14]:
df_from_file.select("InvoiceNo","StockCode").show(10)

+---------+---------+
|InvoiceNo|StockCode|
+---------+---------+
|   536365|   85123A|
|   536365|    71053|
|   536365|   84406B|
|   536365|   84029G|
|   536365|   84029E|
|   536365|    22752|
|   536365|    21730|
|   536366|    22633|
|   536366|    22632|
|   536367|    84879|
+---------+---------+
only showing top 10 rows



In [15]:
df_from_file.sort("InvoiceNo").show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|1.12.2010 08:26|     4,25|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1.12.2010 08:26|     7,65|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:

In [16]:
#sparksın mantıksal planı
df_from_file.sort("InvoiceNo").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [InvoiceNo#56 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(InvoiceNo#56 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#130]
      +- FileScan csv [InvoiceNo#56,StockCode#57,Description#58,Quantity#59,InvoiceDate#60,UnitPrice#61,CustomerID#62,Country#63] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/ege_s/Desktop/pyspark/OnlineRetail.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:string,StockCode:string,Description:string,Quantity:int,InvoiceDate:string,UnitP...




## wordcount örneği

In [17]:
import findspark
findspark.init()

In [18]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [19]:
spark=SparkSession.builder.master("local[4]").appName("wordcount-ornegi").config("spark.executor.memory","4g").config("spark.driver.memory","2g").getOrCreate()

In [21]:
#spark contexti kısaltmada tut
sc=spark.sparkContext

In [22]:
hikaye_df=spark.read.text("C:\\Users\\ege_s\\Desktop\\pyspark\\omer_seyfettin_forsa_hikaye.txt")

In [23]:
hikaye_df.show(3,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                              

In [28]:
#kelime kelime parcalayacagız
from pyspark.sql.functions import explode,split,col

kelimeler=hikaye_df.select(explode(split(col("value")," ")).alias("values"))
kelimeler.show(10)

+-----------+
|     values|
+-----------+
|       Ömer|
|  Seyfettin|
|          -|
|      Forsa|
|           |
|Akdeniz’in,|
|kahramanlık|
|     yuvası|
|     sonsuz|
| ufuklarına|
+-----------+
only showing top 10 rows



In [30]:
from pyspark.sql.functions import desc
kelimeler.groupBy("values").count().orderBy(desc("count")).show(10)

+------+-----+
|values|count|
+------+-----+
|      |   59|
|   bir|   33|
|     –|   31|
|   yıl|    8|
|  diye|    6|
| dedi.|    5|
|  Kırk|    5|
|  onun|    5|
|  Türk|    5|
| doğru|    5|
+------+-----+
only showing top 10 rows



## csv dosyası üzerinden sql sorgusu çalıştırmak